In [1]:
import pandas as pd
from copy import deepcopy

doubleSpaces = [4,6,8]
doubleIndices = [11,12,13]


class Board():
    # creates a standard board
    # first position (0) is not usable
    board = None
    prevNumMoved = None
    totalMoves = 0
    moves = []
    currentFocus = 1

    def __eq__(self, rhs):
        return self.board == rhs.board

    def __init__(self):
        self.board = [0 for i in range(14)]
        for i in range(2,10): self.board[i] = i
        self.board[0] = -1
        self.board[10] = 1
        self.currentFocus = 1
    
    def copy(self):
        c = Board()
        c.board = deepcopy(self.board)
        c.prevNumMoved = self.prevNumMoved
        c.totalMoves = self.totalMoves
        c.moves = deepcopy(self.moves)
        return c

    def display(self):
        a = pd.Series(self.board[1:11])
        b = pd.Series([' ',' ',' ', self.board[11], ' ', self.board[12], ' ', self.board[13], ' ', ' '])
        df = pd.DataFrame([b,a])
        return df.to_string(index=False, header=False)
    
    def isSolved(self):
        return self.board == [-1,1,2,3,4,5,6,7,8,9,0,0,0,0]

# FOR CHECKDOWN AND CHECKUP:
# i :  
    
    def checkDown(self, i):
        try:
            x = self.board.index(i)
            if self.board.index(i) in doubleIndices:
                if self.board[ doubleSpaces[ doubleIndices.index(x) ] ] == 0: 
                    return (True, doubleSpaces[ doubleIndices.index(x) ] )
        except:
            return (False, None)

        return (False, None)

    def checkUp(self, i):
        try:
            x = self.board.index(i)
            if self.board.index(i) in doubleSpaces:
                if self.board[ doubleIndices[ doubleSpaces.index(x) ] ] == 0: 
                    return (True, doubleIndices[ doubleSpaces.index(x) ] )
        except:
            return (False, None)

        return (False, None)

# FOR CHECKLEFT AND CHECKRIGHT:
# i : NUMBER whose left/right position will be checked. Will return a tuple: (is valid move, index checked)
# if i is in doubleIndices, it will automatically return (False, None)

    def checkLeft(self, i):
        try:
            x = self.board.index(i)
            if x-1 not in doubleIndices:
                if self.board[x-1] == 0: return (True, x-1)
        except:
            return (False, None)

        return (False, None)

    def checkRight(self, i):
        try:
            x = self.board.index(i)
            if x+1 not in doubleIndices:
                if self.board[x+1] == 0: return (True, x+1)
        except:
            return (False, None)
        return (False, None)


    def possibleMovesSimple(self, i):
        currentset = []

        up = self.checkUp(i)
        down = self.checkDown(i) 
        left = self.checkLeft(i) 
        right = self.checkRight(i) 

        if up[0]: currentset.append(up[1]); # print('here1')
        if down[0]: currentset.append(down[1]); # print('here2')
        if left[0]: currentset.append(left[1]); # print('here3')
        if right[0]: currentset.append(right[1]); # print('here4')

        return currentset

    # executes a valid move
    # move - tuple (number, new position)
    def move(self, m):
        self.board[self.board.index(m[0])] = 0
        self.board[m[1]] = m[0]
        self.moves.append(m)
        if self.prevNumMoved == m[0]:
            return
        else:
            self.prevNumMoved = m[0]
            self.totalMoves += 1


    # gets a list of all possible game boards that can result from the current in one move (does not add moves that have already been seen)
    def expand(self):
        nodes = []

        runninglist = []
        for i in range(self.currentFocus,10):
            runninglist.append( [i, self.possibleMovesSimple(i)] )

        for item in runninglist:
            if item[1] == []:
                continue
            else: 
                for position in item[1]:
                    temp = self.copy()
                    temp.move((item[0], position))
                    nodes.append(temp)
        return nodes


    # This heuristic will only check to see how far soldier 1 is from its position
    def heuristicA(self):
        distance = 0

        x = self.board.index(1)
        if x in doubleIndices:
            distance += abs(doubleSpaces[doubleIndices.index(x)])
        else:
            distance += abs(x - 1)
        return distance

    
    # This heuristic will sum the distances of each soldier from its respective position
    # Soldier 1's distance matters the most, by a factor of 3
    def heuristicB(self):
        distance = 0

        # if our current focus is in the right spot, lets move our focus
        if (self.board.index(self.currentFocus) == self.currentFocus) and (self.currentFocus < 9): self.currentFocus += 1


        x = self.board.index(self.currentFocus)
        if x in doubleIndices:
            distance += abs(doubleSpaces[doubleIndices.index(x)] - self.currentFocus) + 1
        else: 
            distance += abs(x - self.currentFocus)

    # Improvement mentioned at the end of the report

        indicesOfSpaces = []
        for i in range(1,14):
            if self.board[i] == 0: 
                if i in doubleIndices:
                    distance += abs(doubleSpaces[doubleIndices.index(i)] - self.currentFocus) + 1
                else: 
                    distance += abs(i - self.currentFocus)

        return distance - (6 * self.currentFocus)


In [2]:
def UniformSearch(b):
    nodes = [b]

    num = 0
    while nodes:
        num += 1
        currentnode = nodes[0]
        nodes = nodes[1:]

        if currentnode.isSolved(): return (currentnode, num)

        # QUEUEING FUNCTION
        expantion = currentnode.expand()
        for newnode in expantion:
            nodes.append(newnode)
            
    return (-1, -1)

def aStarHeuristicA(b):
    nodes = [b]

    num = 0
    while nodes:
        num += 1

        currentnode = nodes[0]
        nodes = nodes[1:]
        if currentnode.isSolved(): return (currentnode, num)

        ### START QUEUEING FUNCTION ###

        expantion = currentnode.expand()
        for newnode in expantion:
            nodes.append(newnode)
        nodes = sorted(nodes, key=lambda x: x.heuristicA())

        ### END QUEUEING FUNCTION ###
            
    return (-1, -1)



def aStarHeuristicB(b):
    VISITEDNODES = [b]
    nodes = [VISITEDNODES[0]]

    num = 0
    while nodes:
        num += 1

        currentnode = nodes[0]
        print(currentnode.display(), '       ', currentnode.currentFocus)
        nodes = nodes[1:]
        if currentnode.isSolved(): return (currentnode, num)

        ### START QUEUEING FUNCTION ###

        expantion = currentnode.expand()
        for newnode in expantion:
            if newnode not in VISITEDNODES:
                VISITEDNODES.append(newnode) 
                nodes.append(newnode)
        nodes = sorted(nodes, key=lambda x: x.heuristicB())


        ### END QUEUEING FUNCTION ###
            
    return -1


In [3]:
beginner = Board(); beginner.board =         [0,0,1,2,3,5,6,7,8,9,0,4,0,0]
intermediate = Board(); intermediate.board = [0,2,3,0,0,0,0,5,6,8,9,1,4,7]
advanced = Board()
print(beginner.display(), '\n')
print(intermediate.display(), '\n')
print(advanced.display(), '\n')

      4   0   0    
0 1 2 3 5 6 7 8 9 0 

      1   4   7    
2 3 0 0 0 0 5 6 8 9 

      0   0   0    
0 2 3 4 5 6 7 8 9 1 



In [4]:
# The output following this shows the order in which the nodes were evaluated
results = aStarHeuristicB(advanced)

      7   6   8    
0 0 9 1 0 4 0 2 3 5         1
      7   0   8    
0 0 9 1 0 6 4 2 3 5         1
      7   6   8    
0 9 0 1 0 0 4 2 3 5         1
      0   6   8    
0 0 7 9 1 0 4 2 3 5         1
      9   6   8    
0 7 0 0 0 1 4 2 3 5         1
      9   6   8    
0 0 7 0 1 4 0 2 3 5         1
      9   0   8    
0 0 7 0 1 6 4 2 3 5         1
      9   6   8    
0 7 0 0 1 0 4 2 3 5         1
      9   6   8    
0 0 0 7 1 4 2 0 3 5         1
      0   6   8    
0 9 0 0 7 1 4 2 3 5         1
      9   6   8    
0 0 7 1 0 4 0 2 3 5         1
      9   0   8    
0 0 7 1 0 6 4 2 3 5         1
      9   6   8    
0 7 0 1 0 0 4 2 3 5         1
      7   6   8    
0 9 1 0 0 0 4 2 3 5         1
      9   6   8    
0 7 1 0 0 0 4 2 3 5         1
      0   6   8    
7 0 0 0 9 1 4 2 3 5         1
      0   6   8    
0 7 0 9 0 1 4 2 3 5         1
      0   6   8    
0 9 0 7 0 1 4 2 3 5         1
      7   6   8    
9 0 0 0 0 1 4 2 3 5         1
      7   6   8    
0 0 0 9 1 4 2 3 0 5         1


In [5]:
# Verifying the final board and the total number of nodes evaluated
print(results[0].display(), '\n' ,results[1])

      0   0   0    
1 2 3 4 5 6 7 8 9 0 
 719


In [10]:
# The moves it took to reach the solution
print('It took ', len(results[0].moves), ' moves.')
results[0].moves

It took  103  moves.


[(4, 11),
 (3, 4),
 (2, 3),
 (6, 12),
 (5, 6),
 (3, 5),
 (2, 4),
 (8, 13),
 (9, 8),
 (1, 9),
 (4, 10),
 (2, 11),
 (3, 4),
 (5, 5),
 (7, 6),
 (9, 7),
 (1, 8),
 (4, 9),
 (2, 10),
 (3, 11),
 (5, 4),
 (7, 5),
 (9, 6),
 (1, 7),
 (4, 8),
 (2, 9),
 (3, 10),
 (5, 11),
 (7, 4),
 (9, 5),
 (1, 6),
 (4, 7),
 (2, 8),
 (3, 9),
 (5, 10),
 (7, 11),
 (9, 4),
 (1, 5),
 (9, 3),
 (1, 4),
 (9, 2),
 (1, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (1, 4),
 (9, 3),
 (1, 11),
 (9, 4),
 (9, 5),
 (1, 4),
 (1, 3),
 (1, 2),
 (1, 1),
 (9, 4),
 (9, 11),
 (7, 5),
 (4, 6),
 (2, 7),
 (3, 8),
 (5, 9),
 (9, 10),
 (7, 4),
 (7, 11),
 (4, 5),
 (2, 6),
 (3, 7),
 (5, 8),
 (9, 9),
 (7, 10),
 (4, 4),
 (4, 11),
 (2, 5),
 (2, 4),
 (2, 3),
 (2, 2),
 (3, 6),
 (3, 5),
 (3, 4),
 (3, 3),
 (4, 4),
 (5, 7),
 (5, 6),
 (5, 5),
 (8, 8),
 (8, 7),
 (8, 6),
 (9, 8),
 (7, 9),
 (9, 7),
 (7, 8),
 (7, 13),
 (9, 8),
 (8, 7),
 (9, 9),
 (8, 8),
 (6, 6),
 (9, 10),
 (8, 9),
 (7, 8),
 (7, 7),
 (8, 8),
 (9, 9)]